In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
from tqdm import tqdm
tqdm.monitor_interval = 0

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import json
import pandas as pd
from sklearn.metrics import *

In [4]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
from mobile_net import *

In [7]:
from cifar10 import get_data

In [6]:
PATH = Path("/scratch/arka/miniconda3/external/fastai/courses/dl2/data/cifar10/")
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [8]:
sz = 32
bs = 128

In [9]:
data = get_data(sz, bs, PATH)

In [10]:
md_mbl = mblnetv1(depthwise_block, 
              inc_list=[64, 64, 128, 256], 
              inc_scale = 1, 
              num_blocks_list=[2, 2, 2], 
              stride_list=[1, 2, 2], 
              num_classes=10)

In [11]:
data.trn_ds.transform

[<fastai.transforms.Scale object at 0x7f9e307e45f8>, <fastai.transforms.AddPadding object at 0x7f9e307e4940>, <fastai.transforms.RandomFlip object at 0x7f9e307e4f98>, <fastai.transforms.RandomCrop object at 0x7f9e307e4a90>, <fastai.transforms.Normalize object at 0x7f9e307e47b8>, <fastai.transforms.ChannelOrder object at 0x7f9e307e4d68>]

In [22]:
data.trn_ds.sz

32

In [35]:
np.where(0 == data.trn_ds.y)

(array([   0,    1,    2, ..., 4997, 4998, 4999]),)

In [38]:
set(data.trn_ds.y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [53]:
c1


 476
[torch.LongTensor of size 1]

In [63]:
c0 = 0
c1 = 0
tot = 0
for t in tqdm(range(1000)):
    tmp = torch.LongTensor(1).random_(0, 2)
    c1 += tmp
    c0 += 1 - tmp
    tot += 1
assert (tot == c0 + c1).all()
print(float(c0) / tot)
print(float(c1) / tot)

100%|██████████| 1000/1000 [00:00<00:00, 221206.90it/s]
0.492
0.508


In [23]:
class SiameseDS(BaseDataset):
    def __init__(self, ds):
        self.ds = ds
        super().__init__(ds.transform)
        self.label_set = set(data.trn_ds.y)
        self.l2i = {l: np.where(l == data.trn_ds.y) 
                    for l in self.label_set}
        
    def get_n(self):
        return self.ds.n
    
    def get_c(self):
        return 2
    
    def get_sz(self):
        return ds.sz
    
    def get1item(self, idx):
        targ = torch.LongTensor(1).random_(0, 2)
        if targ == 1